In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import random
from transformers import BertTokenizer, BertModel
import json
import numpy as np
from tqdm import tqdm
import pickle
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
torch.manual_seed(1)

In [2]:
device = "cpu"

In [3]:
# #check if cuda is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device)

In [4]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

#### Data Loading

In [5]:
train_data = json.load(open('../Dataset/NER_train.json', 'r'))
test_data = json.load(open('../Dataset/NER_test.json', 'r'))
val_data = json.load(open('../Dataset/NER_val.json', 'r'))

In [6]:
word_to_idx = pickle.load(open('../Utils/word_to_idx.pkl', 'rb'))

In [7]:
tag_to_ix = pickle.load(open('../Utils/tag_to_ix.pkl', 'rb'))

#### RNN model

In [8]:
class RNN_model(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, target_size, embedding_mat, start_tag, end_tag, tag_to_ix, device='cpu'):
        super(RNN_model, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_mat)).to(device)
        self.rnn = nn.RNN(embedding_dim, hidden_dim).to(device)
        self.hidden2tag = nn.Linear(hidden_dim, target_size).to(device)
        self.start_tag = start_tag
        self.end_tag = end_tag
        self.tag_to_ix = tag_to_ix
        self.target_size = target_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        rnn_out, _ = self.rnn(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(rnn_out.view(len(sentence), -1))
        tag_scores = nn.functional.log_softmax(tag_space, dim=1)
        return tag_scores


#### LSTM Model

In [9]:
class LSTM_model(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, target_size, embedding_mat, start_tag, end_tag, tag_to_ix, device='cpu'):
        super(LSTM_model, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_mat)).to(device)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim).to(device)
        self.hidden2tag = nn.Linear(hidden_dim, target_size).to(device)
        self.start_tag = start_tag
        self.end_tag = end_tag
        self.tag_to_ix = tag_to_ix
        self.target_size = target_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = nn.functional.log_softmax(tag_space, dim=1)
        return tag_scores

#### GRU model

In [10]:
class GRU_model(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, target_size, embedding_mat, start_tag, end_tag, tag_to_ix, device='cpu'):
        super(GRU_model, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_mat)).to(device)
        self.gru = nn.GRU(embedding_dim, hidden_dim).to(device)
        self.hidden2tag = nn.Linear(hidden_dim, target_size).to(device)
        self.start_tag = start_tag
        self.end_tag = end_tag
        self.tag_to_ix = tag_to_ix
        self.target_size = target_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        gru_out, _ = self.gru(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(gru_out.view(len(sentence), -1))
        tag_scores = nn.functional.log_softmax(tag_space, dim=1)
        return tag_scores

#### Embedding mats

In [11]:
bert_embedding_mat = pickle.load(open('../Utils/legal_bert_embedding_mat.pkl', 'rb'))
word2vec_embedding_mat = pickle.load(open('../Utils/word2vec_embedding_mat.pkl', 'rb'))
glove_embedding_mat = pickle.load(open('../Utils/glove_embedding_mat.pkl', 'rb'))

#### Glove + RNN 

In [12]:
rnn_model = RNN_model(len(word_to_idx), 300, 256, len(tag_to_ix), glove_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(rnn_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []


for epoch in range(epochs):
    print("Epoch: ", epoch)
    preds = []
    actuals = []
    train_loss_temp = 0
    val_loss_temp = 0
    rnn_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        rnn_model.zero_grad()
        tag_scores = rnn_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
        actuals.extend(targets.detach().numpy().tolist())
    train_f1_temp = f1_score(actuals, preds, average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp)
    
    with torch.no_grad():
        rnn_model.eval()
        preds = []
        actuals = []
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = rnn_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
            actuals.extend(targets.detach().numpy().tolist())
        val_f1_temp = f1_score(actuals, preds, average='macro')
        val_loss.append(val_loss_temp/len(val_data))
        val_f1.append(val_f1_temp)
    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


100%|██████████| 1416/1416 [00:02<00:00, 688.70it/s]


Train Loss: 0.6250451058513435, Val Loss: 0.527879299745253, Train F1: 0.08089310262663521, Val F1: 0.14462932740032
Epoch:  1


100%|██████████| 1416/1416 [00:02<00:00, 606.62it/s]


Train Loss: 0.4817888299660303, Val Loss: 0.443545947785655, Train F1: 0.22877956783441106, Val F1: 0.27682072301967264
Epoch:  2


...

Train Loss: 0.318555892042339, Val Loss: 0.3418227206814658, Train F1: 0.40763115096515534, Val F1: 0.3973606312352899
Epoch:  8


100%|██████████| 1416/1416 [00:02<00:00, 519.03it/s]


Train Loss: 0.30551658515300406, Val Loss: 0.33521978832420557, Train F1: 0.42308514868059127, Val F1: 0.4152894309524595
Epoch:  9


100%|██████████| 1416/1416 [00:02<00:00, 628.04it/s]

Train Loss: 0.2947103038571048, Val Loss: 0.3307687073831228, Train F1: 0.44120553116551203, Val F1: 0.4253730827131963


In [13]:
torch.save(rnn_model, 'Non Trainable Embeddings/Glove+RNN/model.pt')
torch.save(rnn_model, '../../Deliverables/Task1/Saved Models/t1_RNN_Glove.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Glove+RNN/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Glove+RNN/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Glove+RNN/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Glove+RNN/val_f1.pkl', 'wb'))

#### Word2vec + RNN

In [14]:
rnn_model = RNN_model(len(word_to_idx), 300, 256, len(tag_to_ix), word2vec_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(rnn_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    preds = []
    actuals = []
    train_loss_temp = 0
    val_loss_temp = 0
    rnn_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        rnn_model.zero_grad()
        tag_scores = rnn_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
        actuals.extend(targets.detach().numpy().tolist())
    train_f1_temp = f1_score(actuals, preds, average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp)
    
    with torch.no_grad():
        rnn_model.eval()
        preds = []
        actuals = []
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = rnn_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
            actuals.extend(targets.detach().numpy().tolist())
        val_f1_temp = f1_score(actuals, preds, average='macro')
        val_loss.append(val_loss_temp/len(val_data))
        val_f1.append(val_f1_temp)
    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


  0%|          | 0/8019 [00:00<?, ?it/s]

100%|██████████| 1416/1416 [00:02<00:00, 552.28it/s]


Train Loss: 0.6568109598340905, Val Loss: 0.5123076539343265, Train F1: 0.08876952045176414, Val F1: 0.17728426170253686
Epoch:  1


100%|██████████| 1416/1416 [00:03<00:00, 464.81it/s]


...

100%|██████████| 1416/1416 [00:04<00:00, 318.74it/s]


Train Loss: 0.35709066869295225, Val Loss: 0.3457368135658299, Train F1: 0.36167144078653707, Val F1: 0.3729128094477628
Epoch:  4


100%|██████████| 1416/1416 [00:07<00:00, 190.82it/s]


Train Loss: 0.3315858886363667, Val Loss: 0.33031778971485054, Train F1: 0.3852105191947942, Val F1: 0.39404556612328034
Epoch:  5


  9%|▊         | 687/8019 [00:20<03:09, 38.75it/s]

In [ ]:
torch.save(rnn_model, 'Non Trainable Embeddings/Word2vec+RNN/model.pt')
torch.save(rnn_model, '../../Deliverables/Task1/Saved Models/t1_RNN_Word2Vec.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Word2vec+RNN/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Word2vec+RNN/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Word2vec+RNN/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Word2vec+RNN/val_f1.pkl', 'wb'))

#### Bert + RNN

In [ ]:
rnn_model = RNN_model(len(word_to_idx), 768, 512, len(tag_to_ix), bert_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(rnn_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    preds = []
    actuals = []
    train_loss_temp = 0
    val_loss_temp = 0
    rnn_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        rnn_model.zero_grad()
        tag_scores = rnn_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
        actuals.extend(targets.detach().numpy().tolist())
    train_f1_temp = f1_score(actuals, preds, average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp)
    
    with torch.no_grad():
        rnn_model.eval()
        preds = []
        actuals = []
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = rnn_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
            actuals.extend(targets.detach().numpy().tolist())
        val_f1_temp = f1_score(actuals, preds, average='macro')
        val_loss.append(val_loss_temp/len(val_data))
        val_f1.append(val_f1_temp)
    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


100%|██████████| 1416/1416 [00:04<00:00, 291.18it/s]


Train Loss: 0.7368409467210908, Val Loss: 0.6759775052331177, Train F1: 0.4277597931855873, Val F1: 0.4363632110328191
Epoch:  1


100%|██████████| 1416/1416 [00:04<00:00, 328.16it/s]


Train Loss: 0.6338107338778406, Val Loss: 0.6241690827859828, Train F1: 0.46883327355645327, Val F1: 0.46872462492834777
Epoch:  2


...

Train Loss: 0.47838883303430557, Val Loss: 0.5122087614355598, Train F1: 0.5566179090157893, Val F1: 0.5356131004121726
Epoch:  8


100%|██████████| 1416/1416 [00:04<00:00, 289.39it/s]


Train Loss: 0.4688579626056207, Val Loss: 0.5046447033633473, Train F1: 0.5635292667932726, Val F1: 0.5387408807515905
Epoch:  9


100%|██████████| 1416/1416 [00:04<00:00, 308.58it/s]

Train Loss: 0.45632064041517484, Val Loss: 0.49597533890651274, Train F1: 0.5704571205647169, Val F1: 0.5436559214550242


In [ ]:
torch.save(rnn_model, 'Non Trainable Embeddings/Bert+RNN/model.pt')
torch.save(rnn_model, '../../Deliverables/Task1/Saved Models/t1_RNN_Legal Bert.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Bert+RNN/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Bert+RNN/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Bert+RNN/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Bert+RNN/val_f1.pkl', 'wb'))

#### Glove + GRU

In [ ]:
gru_model = GRU_model(len(word_to_idx), 300, 256, len(tag_to_ix), glove_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(gru_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    preds = []
    actuals = []
    train_loss_temp = 0
    val_loss_temp = 0
    rnn_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        rnn_model.zero_grad()
        tag_scores = rnn_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
        actuals.extend(targets.detach().numpy().tolist())
    train_f1_temp = f1_score(actuals, preds, average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp)
    
    with torch.no_grad():
        rnn_model.eval()
        preds = []
        actuals = []
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = rnn_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
            actuals.extend(targets.detach().numpy().tolist())
        val_f1_temp = f1_score(actuals, preds, average='macro')
        val_loss.append(val_loss_temp/len(val_data))
        val_f1.append(val_f1_temp)
    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


100%|██████████| 1416/1416 [00:07<00:00, 195.19it/s]


Train Loss: 0.6882539816075419, Val Loss: 0.5976716481796387, Train F1: 0.42870465145343867, Val F1: 0.4430916613604604
Epoch:  1


100%|██████████| 1416/1416 [00:06<00:00, 207.10it/s]


Train Loss: 0.5495924759435642, Val Loss: 0.5153488048957726, Train F1: 0.47836394664500254, Val F1: 0.48875410650824846
Epoch:  2


...

Train Loss: 0.3376779318998096, Val Loss: 0.35407042138165523, Train F1: 0.6265041783405458, Val F1: 0.618687691598818
Epoch:  8


100%|██████████| 1416/1416 [00:06<00:00, 219.14it/s]


Train Loss: 0.325041548060872, Val Loss: 0.34575062517185584, Train F1: 0.6359955101217711, Val F1: 0.626447280659448
Epoch:  9


100%|██████████| 1416/1416 [00:07<00:00, 196.38it/s]

Train Loss: 0.31393000045901454, Val Loss: 0.33899746557787597, Train F1: 0.6445698320951921, Val F1: 0.6314090060799529


In [ ]:
torch.save(gru_model, 'Non Trainable Embeddings/Glove+GRU/model.pt')
torch.save(gru_model, '../../Deliverables/Task1/Saved Models/t1_GRU_Glove.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Glove+GRU/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Glove+GRU/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Glove+GRU/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Glove+GRU/val_f1.pkl', 'wb'))

#### Word2vec + GRU

In [ ]:
gru_model = GRU_model(len(word_to_idx), 300, 256, len(tag_to_ix), word2vec_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(gru_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    preds = []
    actuals = []
    train_loss_temp = 0
    val_loss_temp = 0
    rnn_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        rnn_model.zero_grad()
        tag_scores = rnn_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
        actuals.extend(targets.detach().numpy().tolist())
    train_f1_temp = f1_score(actuals, preds, average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp)
    
    with torch.no_grad():
        rnn_model.eval()
        preds = []
        actuals = []
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = rnn_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
            actuals.extend(targets.detach().numpy().tolist())
        val_f1_temp = f1_score(actuals, preds, average='macro')
        val_loss.append(val_loss_temp/len(val_data))
        val_f1.append(val_f1_temp)
    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


100%|██████████| 1416/1416 [00:07<00:00, 182.47it/s]


Train Loss: 0.737562996353968, Val Loss: 0.6298637887660945, Train F1: 0.4227261132052267, Val F1: 0.41879864694068475
Epoch:  1


100%|██████████| 1416/1416 [00:07<00:00, 201.97it/s]


Train Loss: 0.57293946362301, Val Loss: 0.5307543434467072, Train F1: 0.4667838049308789, Val F1: 0.48871960265662123
Epoch:  2


...

Train Loss: 0.3296885823105584, Val Loss: 0.33693415130675036, Train F1: 0.6300532342860933, Val F1: 0.62197680186225
Epoch:  8


100%|██████████| 1416/1416 [00:07<00:00, 198.70it/s]


Train Loss: 0.31441248062531857, Val Loss: 0.325275576554813, Train F1: 0.6397966262046584, Val F1: 0.6283436745609907
Epoch:  9


100%|██████████| 1416/1416 [00:07<00:00, 198.84it/s]

Train Loss: 0.30172139871192627, Val Loss: 0.3159502736656742, Train F1: 0.6484249065592157, Val F1: 0.633736342605233


In [ ]:
torch.save(gru_model, 'Non Trainable Embeddings/Word2vec+GRU/model.pt')
torch.save(gru_model, '../../Deliverables/Task1/Saved Models/t1_GRU_Word2Vec.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Word2vec+GRU/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Word2vec+GRU/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Word2vec+GRU/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Word2vec+GRU/val_f1.pkl', 'wb'))

#### Bert + GRU

In [ ]:
gru_model = GRU_model(len(word_to_idx), 768, 512, len(tag_to_ix), bert_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(gru_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    preds = []
    actuals = []
    train_loss_temp = 0
    val_loss_temp = 0
    rnn_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        rnn_model.zero_grad()
        tag_scores = rnn_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
        actuals.extend(targets.detach().numpy().tolist())
    train_f1_temp = f1_score(actuals, preds, average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp)
    
    with torch.no_grad():
        rnn_model.eval()
        preds = []
        actuals = []
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = rnn_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
            actuals.extend(targets.detach().numpy().tolist())
        val_f1_temp = f1_score(actuals, preds, average='macro')
        val_loss.append(val_loss_temp/len(val_data))
        val_f1.append(val_f1_temp)
    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


  0%|          | 0/8019 [00:00<?, ?it/s]

100%|██████████| 1416/1416 [00:08<00:00, 159.35it/s]


Train Loss: 0.7718177673970524, Val Loss: 0.712550135905523, Train F1: 0.4236934122815464, Val F1: 0.42170276101007276
Epoch:  1


100%|██████████| 1416/1416 [00:11<00:00, 122.34it/s]


...

Train Loss: 0.4821960475611798, Val Loss: 0.5054146561108656, Train F1: 0.5432516232031239, Val F1: 0.5186259280894187
Epoch:  8


100%|██████████| 1416/1416 [00:08<00:00, 158.14it/s]


Train Loss: 0.4682953209257489, Val Loss: 0.4963482150809292, Train F1: 0.5514721365929035, Val F1: 0.5250640637391422
Epoch:  9


100%|██████████| 1416/1416 [00:08<00:00, 166.67it/s]

Train Loss: 0.45596104070301746, Val Loss: 0.4886721015873623, Train F1: 0.5595118671059827, Val F1: 0.5309913574711819


In [ ]:
torch.save(gru_model, 'Non Trainable Embeddings/Bert+GRU/model.pt')
torch.save(gru_model, '../../Deliverables/Task1/Saved Models/t1_GRU_Legal Bert.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Bert+GRU/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Bert+GRU/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Bert+GRU/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Bert+GRU/val_f1.pkl', 'wb'))

#### Glove + LSTM

In [ ]:
lstm_model = LSTM_model(len(word_to_idx), 300, 256, len(tag_to_ix), glove_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(lstm_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    preds = []
    actuals = []
    train_loss_temp = 0
    val_loss_temp = 0
    rnn_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        rnn_model.zero_grad()
        tag_scores = rnn_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
        actuals.extend(targets.detach().numpy().tolist())
    train_f1_temp = f1_score(actuals, preds, average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp)
    
    with torch.no_grad():
        rnn_model.eval()
        preds = []
        actuals = []
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = rnn_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
            actuals.extend(targets.detach().numpy().tolist())
        val_f1_temp = f1_score(actuals, preds, average='macro')
        val_loss.append(val_loss_temp/len(val_data))
        val_f1.append(val_f1_temp)
    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


100%|██████████| 1416/1416 [00:05<00:00, 280.76it/s]


Train Loss: 0.7460598774406, Val Loss: 0.6544797988647942, Train F1: 0.42508995751908224, Val F1: 0.4216943793316018
Epoch:  1


100%|██████████| 1416/1416 [00:05<00:00, 246.85it/s]


Train Loss: 0.611286590423444, Val Loss: 0.5812828869773943, Train F1: 0.44119222107682715, Val F1: 0.45504164818478976
Epoch:  2


...

Train Loss: 0.36223064416297013, Val Loss: 0.37481622979499396, Train F1: 0.6019901011110216, Val F1: 0.596579407022704
Epoch:  8


100%|██████████| 1416/1416 [00:04<00:00, 285.92it/s]


Train Loss: 0.34632849099186774, Val Loss: 0.36451842268310797, Train F1: 0.6137311961358733, Val F1: 0.607048867482402
Epoch:  9


100%|██████████| 1416/1416 [00:04<00:00, 290.04it/s]

Train Loss: 0.332572354783671, Val Loss: 0.35643205488945645, Train F1: 0.624278752916603, Val F1: 0.6170964888249129


In [ ]:
torch.save(lstm_model, 'Non Trainable Embeddings/Glove+LSTM/model.pt')
torch.save(lstm_model, '../../Deliverables/Task1/Saved Models/t1_LSTM_Glove.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Glove+LSTM/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Glove+LSTM/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Glove+LSTM/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Glove+LSTM/val_f1.pkl', 'wb'))

#### Word2vec + LSTM

In [ ]:
lstm_model = LSTM_model(len(word_to_idx), 300, 256, len(tag_to_ix), word2vec_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(lstm_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    preds = []
    actuals = []
    train_loss_temp = 0
    val_loss_temp = 0
    rnn_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        rnn_model.zero_grad()
        tag_scores = rnn_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
        actuals.extend(targets.detach().numpy().tolist())
    train_f1_temp = f1_score(actuals, preds, average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp)
    
    with torch.no_grad():
        rnn_model.eval()
        preds = []
        actuals = []
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = rnn_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
            actuals.extend(targets.detach().numpy().tolist())
        val_f1_temp = f1_score(actuals, preds, average='macro')
        val_loss.append(val_loss_temp/len(val_data))
        val_f1.append(val_f1_temp)
    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


100%|██████████| 1416/1416 [00:04<00:00, 299.94it/s]


Train Loss: 0.8227432623100498, Val Loss: 0.6962273388464362, Train F1: 0.42253654090304393, Val F1: 0.4191522214298386
Epoch:  1


100%|██████████| 1416/1416 [00:04<00:00, 289.97it/s]


Train Loss: 0.6502300609249382, Val Loss: 0.6238453137049699, Train F1: 0.4263505287644238, Val F1: 0.42403403350740615
Epoch:  2


...

Train Loss: 0.37480780907984057, Val Loss: 0.37224403546700824, Train F1: 0.5864893015303219, Val F1: 0.5866231054262977
Epoch:  8


100%|██████████| 1416/1416 [00:05<00:00, 268.26it/s]


Train Loss: 0.3529992197997122, Val Loss: 0.3556284400818052, Train F1: 0.6030219802336886, Val F1: 0.6005365972403784
Epoch:  9


100%|██████████| 1416/1416 [00:06<00:00, 207.98it/s]

Train Loss: 0.3349447936105953, Val Loss: 0.3429008135387505, Train F1: 0.6171922774295108, Val F1: 0.6119369816960177


In [ ]:
torch.save(lstm_model, 'Non Trainable Embeddings/Word2vec+LSTM/model.pt')
torch.save(lstm_model, '../../Deliverables/Task1/Saved Models/t1_LSTM_Word2Vec.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Word2vec+LSTM/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Word2vec+LSTM/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Word2vec+LSTM/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Word2vec+LSTM/val_f1.pkl', 'wb'))

#### Bert + LSTM

In [ ]:
lstm_model = LSTM_model(len(word_to_idx), 768, 512, len(tag_to_ix), bert_embedding_mat, tag_to_ix['START_TAG'], tag_to_ix['END_TAG'], tag_to_ix, device)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(lstm_model.parameters(), lr=0.01)

epochs = 10
train_loss = []
val_loss = []
train_f1 = []
val_f1 = []

for epoch in range(epochs):
    print("Epoch: ", epoch)
    preds = []
    actuals = []
    train_loss_temp = 0
    val_loss_temp = 0
    rnn_model.train()
    for case in tqdm(train_data):
        sentence = prepare_sequence(train_data[case]['text'].split(' '), word_to_idx)
        targets = prepare_sequence(train_data[case]['labels'], tag_to_ix)
        rnn_model.zero_grad()
        tag_scores = rnn_model(sentence)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        train_loss_temp += loss.item()
        preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
        actuals.extend(targets.detach().numpy().tolist())
    train_f1_temp = f1_score(actuals, preds, average='macro')
    train_loss.append(train_loss_temp/len(train_data))
    train_f1.append(train_f1_temp)
    
    with torch.no_grad():
        rnn_model.eval()
        preds = []
        actuals = []
        for case in tqdm(val_data):
            sentence = prepare_sequence(val_data[case]['text'].split(' '), word_to_idx)
            targets = prepare_sequence(val_data[case]['labels'], tag_to_ix)
            tag_scores = rnn_model(sentence)
            loss = loss_function(tag_scores, targets)
            val_loss_temp += loss.item()
            preds.extend(torch.argmax(tag_scores, dim=1).detach().numpy().tolist())
            actuals.extend(targets.detach().numpy().tolist())
        val_f1_temp = f1_score(actuals, preds, average='macro')
        val_loss.append(val_loss_temp/len(val_data))
        val_f1.append(val_f1_temp)
    print(f'Train Loss: {train_loss[-1]}, Val Loss: {val_loss[-1]}, Train F1: {train_f1[-1]}, Val F1: {val_f1[-1]}')

Epoch:  0


100%|██████████| 1416/1416 [00:11<00:00, 127.74it/s]


Train Loss: 0.8090987489394419, Val Loss: 0.7614968724859258, Train F1: 0.4218863639172591, Val F1: 0.41721149948390157
Epoch:  1


100%|██████████| 1416/1416 [00:12<00:00, 115.86it/s]


Train Loss: 0.7181812747980791, Val Loss: 0.707030585882817, Train F1: 0.4275592383267264, Val F1: 0.4230149647993177
Epoch:  2


...

Train Loss: 0.5096635013008204, Val Loss: 0.529840325638073, Train F1: 0.5178209172023773, Val F1: 0.4990178079725728
Epoch:  8


100%|██████████| 1416/1416 [00:09<00:00, 150.17it/s]


Train Loss: 0.49355230407467876, Val Loss: 0.5186011874883851, Train F1: 0.5295591679044606, Val F1: 0.5092878348683654
Epoch:  9


100%|██████████| 1416/1416 [00:09<00:00, 147.17it/s]

Train Loss: 0.47902098420054123, Val Loss: 0.5082034010027067, Train F1: 0.5395419626211806, Val F1: 0.5164417034263564


In [ ]:
torch.save(lstm_model, 'Non Trainable Embeddings/Bert+LSTM/model.pt')
torch.save(lstm_model, '../../Deliverables/Task1/Saved Models/t1_LSTM_Legal Bert.pt')
pickle.dump(train_loss, open('Non Trainable Embeddings/Bert+LSTM/train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('Non Trainable Embeddings/Bert+LSTM/val_loss.pkl', 'wb'))
pickle.dump(train_f1, open('Non Trainable Embeddings/Bert+LSTM/train_f1.pkl', 'wb'))
pickle.dump(val_f1, open('Non Trainable Embeddings/Bert+LSTM/val_f1.pkl', 'wb'))